### Test task
1. В датафреймах (pyspark.sql.DataFrame) заданы продукты, категории и связь между ними. 
2. Одному продукту может соответствовать много категорий, в одной категории может быть много продуктов. 
3. Напишите метод с помощью PySpark, который вернет все продукты с их категориями (датафрейм с набором всех пар «Имя продукта – Имя категории»). 
4. В результирующем датафрейме должны также присутствовать продукты, у которых нет категорий.

In [1]:
from pyspark.sql import SparkSession
import pandas as pd

In [3]:
spark = SparkSession.builder.getOrCreate()

1. Таблица с продуктами:

In [4]:
products = spark.createDataFrame([
    (1, "shampo"),
    (2, "balsam"),
    (3, "conditioner"),
    (4, "oil"),
    (5, "protection"),
], ["product_id", "product_name"])

In [5]:
products.show()

+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|      shampo|
|         2|      balsam|
|         3| conditioner|
|         4|         oil|
|         5|  protection|
+----------+------------+



2. Таблица с категориями

In [6]:
categories = spark.createDataFrame([
    (1, "A"),
    (2, "B"),
    (3, "C"),
    (4, "D"),
    (5, "E"),
    (6, "F")
  
], ["category_id", "category_name"])

categories.show()

+-----------+-------------+
|category_id|category_name|
+-----------+-------------+
|          1|            A|
|          2|            B|
|          3|            C|
|          4|            D|
|          5|            E|
|          6|            F|
+-----------+-------------+



3. Таблица связей продукт-категория (many-to-many)

In [7]:
products_categories = spark.createDataFrame([
    (1, 1),
    (2, 2),
    (2, 3),
    (3, 1),
    (4, 2),
    (4, 6),
    
], ["product_id", "category_id"])

products_categories.show()

+----------+-----------+
|product_id|category_id|
+----------+-----------+
|         1|          1|
|         2|          2|
|         2|          3|
|         3|          1|
|         4|          2|
|         4|          6|
+----------+-----------+



Соединим информацию из таблицы продуктов с таблицей связей:

In [8]:
product_category_outer = products.join(
    products_categories,
    products.product_id ==  products_categories.product_id,
    "left_outer")

In [10]:
product_category_outer.show()

+----------+------------+----------+-----------+
|product_id|product_name|product_id|category_id|
+----------+------------+----------+-----------+
|         1|      shampo|         1|          1|
|         2|      balsam|         2|          3|
|         2|      balsam|         2|          2|
|         3| conditioner|         3|          1|
|         4|         oil|         4|          6|
|         4|         oil|         4|          2|
|         5|  protection|      null|       null|
+----------+------------+----------+-----------+



Соединим полученную таблицу с таблицей категорий:

In [11]:
result = product_category_outer.join(
    categories,
    product_category_outer.category_id == categories.category_id,
    "left_outer"
)

result.show()

+----------+------------+----------+-----------+-----------+-------------+
|product_id|product_name|product_id|category_id|category_id|category_name|
+----------+------------+----------+-----------+-----------+-------------+
|         1|      shampo|         1|          1|          1|            A|
|         2|      balsam|         2|          3|          3|            C|
|         2|      balsam|         2|          2|          2|            B|
|         3| conditioner|         3|          1|          1|            A|
|         4|         oil|         4|          6|          6|            F|
|         4|         oil|         4|          2|          2|            B|
|         5|  protection|      null|       null|       null|         null|
+----------+------------+----------+-----------+-----------+-------------+



In [12]:

result_df = result.select("product_name", 'category_name')
result_df.show()

+------------+-------------+
|product_name|category_name|
+------------+-------------+
|      shampo|            A|
|      balsam|            C|
|      balsam|            B|
| conditioner|            A|
|         oil|            F|
|         oil|            B|
|  protection|         null|
+------------+-------------+



У продукта "protection" изначально не было связи с категорией